# Transfering priority values to the putative protected area network

The original data is a shapefile with the priority of each cell and information of the country. The R script rescales the values within a country from 0 to 255. Then in arcgis pro the shapefile is converted to a raster using the rescaled value. The cell size and extent of the raster is determined by a raster that shows the global putative protected area. Finally, using a constant version of the global putative protected area the pixels that are not part of the global putative protected area are removed. 
#### Note
The R bit can probably be done in arcgis pro using SQL. 

## Processing the data by country in R


```R
library(foreign)
library(plyr)
library(dplyr)
d = read.dbf("Priority_ranks_by_country_20200723/Priority_ranks_by_country_20200723.dbf")

d_range = d %>% 
  group_by(GID_0) %>% 
  summarise(min_pr = min(GID_PR_all), max_pr = max(GID_PR_all), count = n()) %>% 
  right_join(d) %>% 
  mutate(plasma_val = ifelse(count>2,(((GID_PR_all - min_pr) / (max_pr - min_pr)) * 255), 255))   
  write.csv(d_range, "Priority_ranks_by_country_20200723_rescale.csv")
 ```

## From shapefile to raster in ArcGIS pro

In [ ]:
sel = "Priority_ranks_by_country_20200723.shp"
field = "plasma_val"
outRaster = arcpy.FeatureToRaster_conversion(sel, field, outRaster, r"C:\Users\Vizuser\Documents\ArcGIS\Projects\PrioritisationMOL\LP_min_HM_720_WDPAJan2020_20200212.tif")

In [ ]:
raster_rescale = Raster(outRaster)

In [ ]:
output_raster = outRaster * pa_constant_Raster